In [1]:
import numpy as np

In [4]:
x = np.array([1.0, 2.0, 3.0])
y = 2
b = 0.5
e = 1e-5

In [3]:
# если среднее батча = 2, а дисперсия батча = 0,666
b_mean = 2
var = .666

In [7]:
def batch_norm(arr):
    arr_norm = (x - b_mean) / np.sqrt(var**2 + e)
    result = y * arr_norm + b
    return result
    
str(batch_norm(x))

'[-2.50296915  0.5         3.50296915]'

In [8]:
def strange_function(data):

    result = []

    for i in range(len(data)):

        val = 0

        for j in range(i, len(data)):

            val += data[j]

            result.append(val)

    return max(result)

In [9]:
strange_function(x)

np.float64(6.0)

In [11]:
for i in range(0, len(x)):
    print(i)

0
1
2


In [ ]:
import pandas as pd
from typing import Tuple


def foo(df_user_item: pd.DataFrame, df_item_cat: pd.DataFrame, df_item_price: pd.DataFrame) -> pd.DataFrame:
    '''
    Напишите ваш код здесь
    '''
    
    df_merged_tr_and_cat = pd.merge(df_user_item, df_item_cat, on='item_id')
    
    def get_price(row):
        item_prices = df_item_price[df_item_price['item_id'] == row['item_id']]
        valid_prices = item_prices[item_prices['timestamp'] <= row['timestamp']]
        if not valid_prices.empty:
            return valid_prices.nlargest(1, 'timestamp')['price'].iloc[0]
        return None
    

    df_merged_tr_and_cat['price_at_transaction'] = df_merged_tr_and_cat.apply(get_price, axis=1)
    

    result = df_merged_tr_and_cat.groupby(['user_id', 'cat_id'])['price_at_transaction'].mean().reset_index()
    result.columns = ['user_id', 'cat_id', 'avg_price']
    
    
    return result


def make_data() -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    df_user_item = pd.DataFrame(
        {
            "user_id": [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 4, 5, 5, 5, 5],
            "item_id": [1, 2, 3, 4, 1, 2, 3, 4, 1, 1, 3, 4, 4, 4, 4],
            "timestamp": [1, 1, 2, 2, 3, 3, 4, 4, 1, 4, 1, 1, 2, 3, 4],
        }
    )

    df_item_cat = pd.DataFrame(
        {
            "item_id": [1, 2, 3, 4],
            "cat_id": [1, 1, 2, 2],
        }
    )

    df_item_price = pd.DataFrame(
        {
            "item_id": [1, 2, 3, 4, 1, 2, 3, 4],
            "timestamp": [0, 0, 0, 0, 2, 2, 3, 3],
            "price": [100, 200, 300, 400, 500, 600, 700, 800],
        }
    )

    df_user_cat_price_avg = pd.DataFrame(
        {
            "user_id": [1, 1, 2, 2, 3, 4, 5],
            "cat_id": [1, 2, 1, 2, 1, 2, 2],
            "avg_price": [150.0, 350.0, 550.0, 750.0, 300.0, 300.0, 600.0],
        }
    )
    return df_user_item, df_item_cat, df_item_price, df_user_cat_price_avg


def test(func: callable):
    df_user_item, df_item_cat, df_item_price, df_user_cat_price_avg = make_data()
    df_test = func(df_user_item, df_item_cat, df_item_price)
    pd.testing.assert_frame_equal(df_user_cat_price_avg, df_test)


test(foo)